# Ансамбли

1111


In [6]:
import sys

from tensorflow.keras import datasets, layers, models, losses
import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
import matplotlib.pyplot as plt
from livelossplot import PlotLossesKeras
import tensorflow_addons as tfa

sys.path.append('../../../')

from core.datasets import get_ds
from core.make_answer import make_ans_file

In [7]:
transformer = tv.transforms.Compose([
    tv.transforms.ColorJitter(brightness=.2, hue=0.2, contrast=0.5),
    tv.transforms.RandomAffine(degrees=(-10, 10), translate=(0, 0.1), scale=(0.85, 1)),
    tv.transforms.RandomPerspective(distortion_scale=0.2, p=0.7),
])

In [8]:
train_ds, val_ds, test_ds = get_ds('repaired_data_train', 'repaired_data_test', transform=transformer, one_hot=True, back=3, val_size=0.9)
val_ds_x = np.array([i[0] for i in val_ds.unbatch().as_numpy_iterator()])
val_ds_y = np.array([i[1] for i in val_ds.unbatch().as_numpy_iterator()])
val_ds_y = np.argmax(val_ds_y, axis=1)

In [9]:
model_1 = tf.keras.models.load_model(f'../../../models/checkpoints/model_g_1_categorical_accuracy.h5')
model_3 = tf.keras.models.load_model(f'../../../models/checkpoints/model_g_3_categorical_accuracy.h5')
model_4 = tf.keras.models.load_model(f'../../../models/checkpoints/model_g_4_avg_categorical_accuracy.h5')

models = [model_1, model_3, model_4]

- Bagging (пакетирование) голосование, предпочтительна система наподобие земских соборов с представителями разных сословий нейронных сетей.

In [10]:
def get_bagging_pred(models, x):
    pred_summ = np.full((len(x), 10), 0.)
    for m in models:
        pred_summ += m.predict(x,  verbose=False)
    pred_summ = np.array([np.argmax(i) for i in pred_summ])
    return pred_summ

def get_accuracy(y_pred, y_true):
    return round(np.sum(y_pred == y_true) / len(y_pred), 4)

y_pred_en = get_bagging_pred(models=models, x=val_ds_x)


print("ACCURACY SCORE")
print("model G1:", get_accuracy(np.argmax(model_1.predict(val_ds_x, verbose=False), axis=1), val_ds_y))
print("model G3:", get_accuracy(np.argmax(model_3.predict(val_ds_x, verbose=False), axis=1), val_ds_y))
print("model C1:", get_accuracy(np.argmax(model_4.predict(val_ds_x, verbose=False), axis=1), val_ds_y))

print("ensemble bagging:", get_accuracy(y_pred_en, val_ds_y))

ACCURACY SCORE
model G1: 0.9802
model G3: 0.9703
model C1: 0.9701
ensemble bagging: 0.9866
